# Cài đặt thư viện

In [1]:
!pip install requests beautifulsoup4
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 19.2 MB/s eta 0:00:00


In [17]:
import pandas as pd

# Thiết lập để hiển thị toàn bộ DataFrame (tắt giới hạn hàng và cột)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Lấy URL

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# URL của trang web
url = 'https://xebuyt.net/tuyen-xe-buyt'

# Gửi request để lấy nội dung trang
response = requests.get(url)
response.raise_for_status()  # Kiểm tra lỗi nếu có

# Parse HTML với BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Tìm div chứa danh sách tuyến
div_result = soup.find('div', id='divResult')

# Tìm tất cả các thẻ <a> với class 'cms-button'
route_links = div_result.find_all('a', class_='cms-button')

# Danh sách để lưu dữ liệu
data = []

# Duyệt qua từng link
for link in route_links:
    # Lấy href
    href = link.get('href')
    full_url = 'https://xebuyt.net' + href if href.startswith('/') else href

    # Lấy số tuyến từ span.icon
    route_number = link.find('span', class_='icon').get_text(strip=True)

    # Lấy tên tuyến từ div.routetrip
    route_name = link.find('div', class_='routetrip').get_text(strip=True)

    # Thêm vào danh sách
    data.append({
        'Route_Number': route_number,
        'Route_Name': route_name,
        'URL': full_url
    })

# Lưu vào file CSV
filename = 'tuyen_xe_buyt.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Route_Number', 'Route_Name', 'URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(data)

print(f'Dữ liệu đã được lưu vào file {filename}. Tổng số tuyến: {len(data)}')

# Hiển thị dưới dạng DataFrame (comment nếu không cần)
df = pd.DataFrame(data)
print(df)

Dữ liệu đã được lưu vào file tuyen_xe_buyt.csv. Tổng số tuyến: 140
    Route_Number                                         Route_Name  \
0             01                              Bến Thành- BX Chợ Lớn   
1             02                             Bến Thành- BX Miền Tây   
2             03                               Bến Thành- Thạnh Lộc   
3             04                      Bến Thành- Cộng Hòa- An Sương   
4             05                          Bến xe Chợ Lớn - Biên Hòa   
..           ...                                                ...   
135          149        Công viên 23/9 - Khu dân cư Bình Hưng Hòa   
136          150                     Bến xe Chợ Lớn - Ngã 3 Tân Vạn   
137          151                  Bến xe Miền Tây - Bến xe An Sương   
138          152  Khu dân cư Trung Sơn - Bến Thành - Sân bay T...   
139         C010                         Phổ Quang - KCN Hiệp Phước   

                                                   URL  
0     https://xebuyt.ne

# Lấy thông tin chuyến xe

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
df = pd.read_csv(csv_filename)

# Danh sách để lưu dữ liệu mở rộng
extended_data = []

# Headers cho CSV mới
headers = [
    'Route_Number', 'Route_Name', 'URL',
    'Operator', 'Operating_Type', 'Distance', 'Vehicle_Type', 'Operating_Hours',
    'Ticket_Price', 'Number_of_Trips', 'Trip_Time', 'Frequency'
]

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"Đang scrape tuyến {route_number}: {route_name}")

    try:
        # Gửi request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm div rouInfo và table
        rou_info_div = soup.find('div', id='rouInfo')
        if not rou_info_div:
            print(f"Không tìm thấy div#rouInfo cho tuyến {route_number}")
            continue

        table = rou_info_div.find('table', class_='tbl100')
        if not table:
            print(f"Không tìm thấy table cho tuyến {route_number}")
            continue

        # Khởi tạo giá trị mặc định
        operator = ''
        operating_type = ''
        distance = ''
        vehicle_type = ''
        operating_hours = ''
        ticket_price = ''
        number_of_trips = ''
        trip_time = ''
        frequency = ''

        # Duyệt qua các hàng trong table
        rows = table.find('tbody').find_all('tr')
        for tr in rows:
            tds = tr.find_all('td')
            if len(tds) < 2:
                continue

            label_td = tds[0].get_text(strip=True)
            content = tds[1].get_text(separator='\n').strip()

            # Xử lý Đơn vị đảm nhận (có thể ở hàng riêng)
            if 'Đơn vị đảm nhận' in label_td:
                operator = content

            # Bỏ qua hàng spacing
            if not label_td and not content:
                continue

            # Xử lý ul lists ở hàng cuối (cột 2 và 3)
            if len(tds) >= 3 and not label_td:  # Hàng có ul lists
                left_ul = tds[1].find('ul')
                right_ul = tds[2].find('ul') if len(tds) > 2 else None

                if left_ul:
                    left_items = [li.get_text(strip=True) for li in left_ul.find_all('li')]
                    for item in left_items:
                        if 'Loại hình hoạt động' in item:
                            operating_type = item.replace('Loại hình hoạt động:', '').strip()
                        elif 'Cự ly' in item:
                            distance = item.replace('Cự ly:', '').strip()
                        elif 'Loại xe' in item:
                            vehicle_type = item.replace('Loại xe:', '').strip()
                        elif 'Thời gian hoạt động' in item:
                            operating_hours = item.replace('Thời gian hoạt động:', '').strip()

                if right_ul:
                    right_items = [li.get_text(separator='\n').strip() for li in right_ul.find_all('li')]
                    for item in right_items:
                        if 'Giá vé' in item:
                            ticket_price = item.replace('Giá vé:', '').strip()
                        elif 'Số chuyến' in item:
                            number_of_trips = item.replace('Số chuyến:', '').strip()
                        elif 'Thời gian chuyến' in item:
                            trip_time = item.replace('Thời gian chuyến:', '').strip()
                        elif 'Giãn cách chuyến' in item:
                            frequency = item.replace('Giãn cách chuyến:', '').strip()

        # Thêm vào danh sách
        extended_data.append({
            'Route_Number': route_number,
            'Route_Name': route_name,
            'URL': url,
            'Operator': operator,
            'Operating_Type': operating_type,
            'Distance': distance,
            'Vehicle_Type': vehicle_type,
            'Operating_Hours': operating_hours,
            'Ticket_Price': ticket_price,
            'Number_of_Trips': number_of_trips,
            'Trip_Time': trip_time,
            'Frequency': frequency
        })

    except Exception as e:
        print(f"Lỗi khi scrape tuyến {route_number}: {e}")
        # Vẫn thêm dữ liệu với giá trị rỗng
        extended_data.append({
            'Route_Number': route_number,
            'Route_Name': route_name,
            'URL': url,
            'Operator': '',
            'Operating_Type': '',
            'Distance': '',
            'Vehicle_Type': '',
            'Operating_Hours': '',
            'Ticket_Price': '',
            'Number_of_Trips': '',
            'Trip_Time': '',
            'Frequency': ''
        })

    # Delay để tránh overload server
    # time.sleep(1)

# Tạo DataFrame và lưu vào CSV mới
extended_df = pd.DataFrame(extended_data)
output_filename = 'tuyen_xe_buyt_detailed.csv'
extended_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f'Dữ liệu chi tiết đã được lưu vào file {output_filename}. Tổng số tuyến: {len(extended_data)}')

# Hiển thị DataFrame (comment nếu không cần)
print(extended_df.head())

Đang scrape tuyến 01: Bến Thành- BX Chợ Lớn
Đang scrape tuyến 02: Bến Thành- BX Miền Tây
Đang scrape tuyến 03: Bến Thành- Thạnh Lộc
Đang scrape tuyến 04: Bến Thành- Cộng Hòa- An Sương
Đang scrape tuyến 05: Bến xe Chợ Lớn - Biên Hòa
Đang scrape tuyến 06: Bến xe Chợ Lớn- Đại học Nông Lâm
Đang scrape tuyến 07: Bến xe Chợ Lớn- Gò vấp
Đang scrape tuyến 08: Bến xe Quận 8- Đại học Quốc Gia
Đang scrape tuyến 09: Chợ Lớn - Bình Chánh - Hưng Long
Đang scrape tuyến 10: Đại học Quốc Gia- Bến xe Miền Tây
Đang scrape tuyến 11: Bến Thành- Đầm Sen
Đang scrape tuyến 12: Bến Thành - Thác Giang Điền
Đang scrape tuyến 13: Bến Thành- Bến xe Củ Chi
Đang scrape tuyến 14: Miền Đông- 3 tháng 2- Miền Tây
Đang scrape tuyến 15: Bến Phú Định- Đầm Sen
Đang scrape tuyến 16: Bến xe Chợ Lớn - Bến xe Tân Phú
Đang scrape tuyến 17: Bến Xe Chợ Lớn - ĐH Sài Gòn - KCX Tân Thuận
Đang scrape tuyến 18: Bến Thành - Chợ Hiệp Thành
Đang scrape tuyến 19: Bến Thành - Khu Chế Xuất Linh Trung - Đại Học Quốc Gia
Đang scrape tuyến 2

# Lấy lộ trình

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time  # Để delay tránh spam

# Đọc CSV
df_links = pd.read_csv('tuyen_xe_buyt.csv')

all_data = []

for index, row in df_links.iterrows():
    url = row['URL']
    route_num = row['Route_Number']
    route_name = row['Route_Name']
    print(f"Crawling {route_num} - {route_name} ({url})...")

    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Lỗi fetch {url}: {response.status_code}")
            continue
        soup = BeautifulSoup(response.content, 'html.parser')

        # Ưu tiên tab lotrinh cho trang phức tạp
        lotrinh = soup.find(id='lotrinh')
        if lotrinh:
            sections = lotrinh.find_all('h3')  # Các hướng trong accordion
            for h3 in sections:
                direction = h3.get_text(strip=True)
                if 'Đi' in direction:
                    div = h3.find_next('div', class_='ui-accordion-content')
                    if div:
                        table = div.find('table')
                        if table:
                            stations = []
                            rows = table.find('tbody').find_all('tr') if table.find('tbody') else table.find_all('tr')
                            for row_tr in rows:
                                tds = row_tr.find_all('td')
                                if len(tds) >= 2:
                                    order = tds[0].get_text(strip=True)
                                    if order not in ['A', 'B']:  # Skip A/B, lấy trạm giữa
                                        station = tds[1].get_text(strip=True).strip()
                                        if station and station not in stations:  # Unique
                                            stations.append(station)
                            if stations:
                                for station in stations:
                                    all_data.append({
                                        'Route_Number': route_num,
                                        'Route_Name': route_name,
                                        'Huong': direction,
                                        'Tram': station
                                    })
        else:
            # Fallback cho trang đơn giản: Tìm h3 như 'Đi ...' rồi table next
            h3s = soup.find_all('h3')
            for h3 in h3s:
                direction = h3.get_text(strip=True)
                if 'Đi' in direction:
                    table = h3.find_next('table')
                    if table:
                        stations = []
                        rows = table.find_all('tr')
                        for row_tr in rows:
                            tds = row_tr.find_all(['td', 'th'])
                            # Lọc text: Ghép td không phải số/A/B
                            station_parts = [td.get_text(strip=True) for td in tds if td.get_text(strip=True) and not td.get_text(strip=True).isdigit() and td.get_text(strip=True) not in ['A', 'B']]
                            station = ' '.join(station_parts).strip()
                            if station and station not in stations:
                                stations.append(station)
                        if stations:
                            for station in stations:
                                all_data.append({
                                    'Route_Number': route_num,
                                    'Route_Name': route_name,
                                    'Huong': direction,
                                    'Tram': station
                                })

        # time.sleep(1)  # Delay 1s
    except Exception as e:
        print(f"Lỗi xử lý {url}: {e}")
        continue

# Tạo DF, drop duplicate toàn bộ
df_lo_trinh = pd.DataFrame(all_data).drop_duplicates()
print(df_lo_trinh.head(20).to_string(index=False))  # Preview top 20 rows
df_lo_trinh.to_csv('lo_trinh_crawl.csv', index=False, encoding='utf-8-sig')  # utf-8-sig cho Excel OK
print(f"\nCrawl xong! Tổng {len(df_lo_trinh)} records unique, lưu ở lo_trinh_crawl.csv")

Crawling 01 - Bến Thành- BX Chợ Lớn (https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html)...
Crawling 02 - Bến Thành- BX Miền Tây (https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html)...
Crawling 03 - Bến Thành- Thạnh Lộc (https://xebuyt.net/ben-thanh-thanh-loc.html)...
Crawling 04 - Bến Thành- Cộng Hòa- An Sương (https://xebuyt.net/ben-thanh-cong-hoa-an-suong.html)...
Crawling 05 - Bến xe Chợ Lớn - Biên Hòa (https://xebuyt.net/ben-xe-cho-lon-bien-hoa.html)...
Crawling 06 - Bến xe Chợ Lớn- Đại học Nông Lâm (https://xebuyt.net/ben-xe-cho-lon-dai-hoc-nong-lam.html)...
Crawling 07 - Bến xe Chợ Lớn- Gò vấp (https://xebuyt.net/ben-xe-cho-lon-go-vap.html)...
Crawling 08 - Bến xe Quận 8- Đại học Quốc Gia (https://xebuyt.net/ben-xe-quan-8-dai-hoc-quoc-gia.html)...
Crawling 09 - Chợ Lớn - Bình Chánh - Hưng Long (https://xebuyt.net/cho-lon-binh-chanh-hung-long.html)...
Crawling 10 - Đại học Quốc Gia- Bến xe Miền Tây (https://xebuyt.net/dai-hoc-quoc-gia-ben-xe-mien-tay.html)...
Crawling 11 - Bến Thành

# Lấy lộ trình - fixed

**Lần 1 - tạm ổn**

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
df = pd.read_csv(csv_filename)

# Danh sách để lưu dữ liệu lộ trình (mỗi stop là một row)
route_stops_data = []

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"Đang scrape lộ trình tuyến {route_number}: {route_name}")

    try:
        # Gửi request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm div lotrinh
        lotrinh_div = soup.find('div', id='lotrinh')
        if not lotrinh_div:
            print(f"Không tìm thấy div#lotrinh cho tuyến {route_number}")
            continue

        # Tìm tất cả accordion headers và contents
        accordion = lotrinh_div.find('div', id='accordion')
        if not accordion:
            print(f"Không tìm thấy accordion cho tuyến {route_number}")
            continue

        # Tìm tất cả h3 (headers cho hướng)
        headers = accordion.find_all('h3', class_='ui-accordion-header')

        for header in headers:
            # Lấy tên hướng (direction)
            direction = header.get_text(strip=True)

            # Tìm div content tương ứng (thường là sibling next div)
            content_div = header.find_next_sibling('div', class_='ui-accordion-content')
            if not content_div:
                continue

            # Tìm table trong content
            table = content_div.find('table')
            if not table:
                continue

            # Tìm tất cả rows trong tbody
            rows = table.find('tbody').find_all('tr', class_='rowStop')

            # Duyệt qua từng stop
            for row in rows:
                tds = row.find_all('td')
                if len(tds) < 2:
                    continue

                # Lấy thứ tự stop (A, 1, 2, ..., B)
                order_no_td = tds[0].find('div')
                stop_order = order_no_td.get_text(strip=True) if order_no_td else ''

                # Lấy tên stop
                stop_name_td = tds[1]
                stop_name = stop_name_td.get_text(strip=True)

                # Thêm vào danh sách
                route_stops_data.append({
                    'Route_Number': route_number,
                    'Route_Name': route_name,
                    'URL': url,
                    'Direction': direction,
                    'Stop_Order': stop_order,
                    'Stop_Name': stop_name
                })

    except Exception as e:
        print(f"Lỗi khi scrape lộ trình tuyến {route_number}: {e}")

    # Delay để tránh overload server
    # time.sleep(1)

# Tạo DataFrame và lưu vào CSV mới
route_stops_df = pd.DataFrame(route_stops_data)
output_filename = 'tuyen_xe_buyt_lotrinh.csv'
route_stops_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f'Dữ liệu lộ trình đã được lưu vào file {output_filename}. Tổng số stops: {len(route_stops_data)}')

# Hiển thị DataFrame (comment nếu không cần)
route_stops_df.head(10)

Đang scrape lộ trình tuyến 01: Bến Thành- BX Chợ Lớn
Đang scrape lộ trình tuyến 02: Bến Thành- BX Miền Tây
Đang scrape lộ trình tuyến 03: Bến Thành- Thạnh Lộc
Đang scrape lộ trình tuyến 04: Bến Thành- Cộng Hòa- An Sương
Đang scrape lộ trình tuyến 05: Bến xe Chợ Lớn - Biên Hòa
Đang scrape lộ trình tuyến 06: Bến xe Chợ Lớn- Đại học Nông Lâm
Đang scrape lộ trình tuyến 07: Bến xe Chợ Lớn- Gò vấp
Đang scrape lộ trình tuyến 08: Bến xe Quận 8- Đại học Quốc Gia
Đang scrape lộ trình tuyến 09: Chợ Lớn - Bình Chánh - Hưng Long
Đang scrape lộ trình tuyến 10: Đại học Quốc Gia- Bến xe Miền Tây
Đang scrape lộ trình tuyến 11: Bến Thành- Đầm Sen
Đang scrape lộ trình tuyến 12: Bến Thành - Thác Giang Điền
Đang scrape lộ trình tuyến 13: Bến Thành- Bến xe Củ Chi
Đang scrape lộ trình tuyến 14: Miền Đông- 3 tháng 2- Miền Tây
Đang scrape lộ trình tuyến 15: Bến Phú Định- Đầm Sen
Đang scrape lộ trình tuyến 16: Bến xe Chợ Lớn - Bến xe Tân Phú
Đang scrape lộ trình tuyến 17: Bến Xe Chợ Lớn - ĐH Sài Gòn - KCX Tân

""


**Lần 2**

# Lấy biểu đồ giờ

In [22]:
import requests
import pandas as pd
import json
import time  # Để delay tránh spam

# Đọc CSV (cấu trúc: Route_Number, Route_Name, URL)
df_routes = pd.read_csv('tuyen_xe_buyt.csv')
print(f"Đọc CSV OK: {len(df_routes)} routes")

# Base URL API
base_url = "https://api.xebuyt.net/businfo/"

# Dict để lưu data: {route_num: {vars: [], stops: {var_id: []}, paths: {var_id: {lat: [], lng: []}}}}
all_data = {}

for index, row in df_routes.iterrows():
    route_num = row['Route_Number'].zfill(2)  # Đảm bảo '01' thay vì 1
    route_name = row['Route_Name']
    print(f"\n--- Crawling tuyến {route_num} - {route_name} ---")

    try:
        # API 1: Get vars by route (e.g., /getvarsbyroute/1_1 cho tuyến 01)
        vars_url = f"{base_url}getvarsbyroute/{route_num}_1"
        response_vars = requests.get(vars_url, timeout=10)
        if response_vars.status_code != 200:
            print(f"Lỗi fetch vars {vars_url}: {response_vars.status_code}")
            continue
        vars_data = response_vars.json()
        print(f"Vars extracted: {len(vars_data)} variants")
        all_data[route_num] = {'vars': vars_data, 'stops': {}, 'paths': {}}

        # Loop qua từng variant (thường 2: đi và về)
        for var in vars_data:
            var_id = var['RouteVarId']  # '1' cho đi, '2' cho về
            direction = var['RouteVarName']
            print(f"  Processing variant {var_id}: {direction}")

            # API 2: Get stops by var (e.g., /getstopsbyvar/1_1/1)
            stops_url = f"{base_url}getstopsbyvar/{route_num}_1/{var_id}"
            response_stops = requests.get(stops_url, timeout=10)
            if response_stops.status_code == 200:
                stops_data = response_stops.json()
                print(f"    Stops extracted: {len(stops_data)} stops")
                all_data[route_num]['stops'][var_id] = stops_data
            else:
                print(f"    Lỗi fetch stops {stops_url}: {response_stops.status_code}")

            # API 3: Get paths by var (e.g., /getpathsbyvar/1_1/1)
            paths_url = f"{base_url}getpathsbyvar/{route_num}_1/{var_id}"
            response_paths = requests.get(paths_url, timeout=10)
            if response_paths.status_code == 200:
                paths_data = response_paths.json()
                print(f"    Paths extracted: {len(paths_data.get('lat', []))} points")
                all_data[route_num]['paths'][var_id] = paths_data
            else:
                print(f"    Lỗi fetch paths {paths_url}: {response_paths.status_code}")

        time.sleep(1)  # Delay
    except Exception as e:
        print(f"Lỗi tổng {route_num}: {e}")
        continue

# Lưu full data vào JSON
with open('api_crawl_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)
print("\nLưu data full vào api_crawl_data.json")

# Nếu muốn CSV flat cho stops (dễ xem)
stops_list = []
for route_num, data in all_data.items():
    for var_id, stops in data['stops'].items():
        for stop in stops:
            stops_list.append({
                'Route_Number': route_num,
                'RouteVarId': var_id,
                'Stop_Name': stop['Name'],
                'Stop_Code': stop['Code'],
                'Lat': stop['Lat'],
                'Lng': stop['Lng'],
                'Address': f"{stop['AddressNo']} {stop['Street']}",
                'Routes': stop['Routes'],
                'Status': stop['Status'],
                'StopType': stop['StopType']
            })

df_stops = pd.DataFrame(stops_list).drop_duplicates()
df_stops.to_csv('stops_crawl.csv', index=False, encoding='utf-8-sig')
print(f"Lưu stops vào stops_crawl.csv (Tổng {len(df_stops)} records)")

# Tương tự cho paths nếu cần, nhưng paths là coords dài, nên giữ JSON
print("Crawl xong! Check files để xem data.")

Đọc CSV OK: 140 routes

--- Crawling tuyến 01 - Bến Thành- BX Chợ Lớn ---
Vars extracted: 2 variants
  Processing variant 1: Lượt đi: Bến Thành - BX Chợ Lớn
    Stops extracted: 29 stops
    Paths extracted: 99 points
  Processing variant 2: Lượt về: BX Chợ Lớn - Bến Thành
    Stops extracted: 35 stops
    Paths extracted: 141 points

--- Crawling tuyến 02 - Bến Thành- BX Miền Tây ---
Vars extracted: 2 variants
  Processing variant 3: Lượt đi: Bến Thành - BX Miền Tây
    Stops extracted: 43 stops
    Paths extracted: 167 points
  Processing variant 4: Lượt về: BX Miền Tây - Bến Thành
    Stops extracted: 37 stops
    Paths extracted: 166 points

--- Crawling tuyến 03 - Bến Thành- Thạnh Lộc ---
Vars extracted: 2 variants
  Processing variant 5: Lượt đi: Bến Thành - Thạnh Lộc
    Stops extracted: 48 stops
    Paths extracted: 203 points
  Processing variant 6: Lượt về: Thạnh Lộc - Bến Thành
    Stops extracted: 48 stops
    Paths extracted: 193 points

--- Crawling tuyến 04 - Bến Thành- 